In [1]:
from solcore import si, material
from solcore.structure import Layer, Structure, Junction
from solcore.solar_cell import SolarCell
import solcore.quantum_mechanics as QM
import solcore.poisson_drift_diffusion as PDD
import numpy as np
from solcore.light_source import LightSource
from solcore.solar_cell_solver import solar_cell_solver
import matplotlib.pyplot as plt
import os
import shutil

In [ ]:
wl = np.linspace(350, 1200, 401) * 1e-9
light_source = LightSource(
    source_type="standard",
    version="AM1.5g",
    x=wl,
    output_units="photon_flux_per_m",
    concentration=1,
)
bulk = material("GaAs")(T=293, strained=False)
barrier = material("GaAsP")(T=293, P=0.1, strained=True)

# As well as some of the layers
top_layer = Layer(width=si("30nm"), material=barrier)
inter = Layer(width=si("3nm"), material=bulk)
barrier_layer = Layer(width=si("5nm"), material=barrier)
bottom_layer = top_layer
wl = np.linspace(350, 1050, 301) * 1e-9

InSb = material('InSb')(T=293, strained=False)
GaSb = material('GaSb')(T=293, strained=False)
buffer = material('AlGaAs')(T=293, Al=0.3)
capping = material('GaAs')(T=293)
n_GaAs = material('GaAs')(T=293, Nd=1e15)
p_GaAs = material('GaAs')(T=293, Na=1e15)
# We create the QW material at the given composition
QW = material("InGaAs")(T=293, In=0.15, strained=True)

n_layer = Layer(width=250e-9, material=n_GaAs)
QW1 = Layer(width=5e-9, material=InSb)
Capping_layer = Layer(width=80e-9, material=capping)
QW2 = Layer(width=5e-9, material=GaSb)
p_layer = Layer(width=400e-9, material=p_GaAs)
test_ver1 = Structure([n_layer, QW1, Capping_layer, QW2, p_layer], substrate=bulk)
test_ver2 = Structure([n_layer, QW1, Capping_layer, p_layer], substrate=bulk)

In [ ]:
T = 293
    n_GaAs = material("GaAs")(T=T, Nd=1e24)
    p_GaAs = material("GaAs")(T=T, Na=8e22)
    QWmat = material("InGaAs")(T=T, In=0.2, strained=True)
    Bmat = material("GaAsP")(T=T, P=0.1, strained=True)
    i_GaAs = material("GaAs")(T=T)
    i_GaAs_dope = material("GaAs")(T=T, Na= 1e14)

    struc = Structure([Layer(width=50e-9, material=n_GaAs, role="Emitter")]+
                      [Layer(width=barrier, material=Bmat, role="barrier")] +
                      + 10 * [Layer(width=barrier, material=Bmat, role="barrier"),
                              Layer(width=interlayer, material=i_GaAs_dope, role="interlayer"),
                              Layer(width=dot, material=QWmat, role="well"),
                              Layer(width=interlayer, material=i_GaAs_dope, role="interlayer"),
                              Layer(width=barrier, material=Bmat, role="barrier")] +
                      # +10 * [Layer(width=10e-9, material=Bmat, role="barrier"),
                             # Layer(width=2e-9, material=i_GaAs, role="well"),
                             # Layer(width=7e-9, material=QWmat, role="well"),
                             # Layer(width=2e-9, material=i_GaAs, role="well"),
                             # Layer(width=10e-9, material=Bmat, role="barrier"),] +
                      [Layer(width=barrier, material=Bmat, role="barrier")] +
                      [Layer(width=50e-9, material=p_GaAs, role="Base")],
                      substrate=i_GaAs)
    # print(struc)
    output_2 = QM.schrodinger(struc, quasiconfined=0.05, graphtype='potentialsLDOS', num_eigenvalues=200, show=True)
    # print(output_2)